In [1]:
# when using jupyter lab its important install your kernel and library appropriately so you won't run into errors
# anaconda tutorial
# conda env create --name MyEnvName python= -c conda-forge
# conda activate MyEnvName
# conda install -c conda-forge jupyterlab
# python -m ipykernel install --user --name=MyEnvName
# jupyter lab

In [2]:
# initialize and import required dependencies and library
import pandas as pd
from bs4 import BeautifulSoup

In [3]:
from datetime import datetime, timedelta
import requests

In [4]:
# get request and response of the url
url = "https://www.ccny.cuny.edu/registrar/fall"
r = requests.get(url)

# utilize beautiful soup library to extract html
soup = BeautifulSoup(r.text, 'html.parser')
gethtml = soup.prettify()
# print(gethtml)

In [5]:
calender = soup.find('table')

In [6]:
# we now must iterate through the row which can be defined by tr. find_all will provide list of all tr elements
rows = calender.find_all('tr')

In [7]:
# initalize a list that is to be used for the dataframe later
calenderData = []
for row in rows:
    cols = row.find_all('td')
    if len(cols) >= 3:
        date_str = cols[0].text.strip()
        dow_str = cols[1].text.strip()
        text_str = cols[2].text.strip()

        # Clean unwanted characters in the text description
        text_str = text_str.replace('\n', ' ').replace('\t', ' ').strip()

        # Handle single dates and date ranges
        if "-" in date_str:
            # Handle a date range
            date_range = date_str.split(' - ')
            start_date_str = date_range[0] + ", 2021"
            month = start_date_str.split(' ')[0]
            end_date_str = month + ' ' + date_range[1] + ", 2021"

            # Convert to datetime objects
            start_date = datetime.strptime(start_date_str, "%B %d, %Y").date()
            end_date = datetime.strptime(end_date_str, "%B %d, %Y").date()

            # Iterate through the range and append each date
            curr_date = start_date
            while curr_date <= end_date:
                calenderData.append({
                    "date": curr_date,
                    "dow": curr_date.strftime('%A'),
                    "text": text_str
                })
                curr_date += timedelta(days=1)

        else:
            # Handle a single date
            if "2022" in date_str:
                # Date is in 2022
                date = datetime.strptime(date_str, "%B %d, %Y").date()
            else:
                # Date is in 2021
                date_str += ", 2021"
                date = datetime.strptime(date_str, "%B %d, %Y").date()

            calenderData.append({
                "date": date,
                "dow": dow_str,
                "text": text_str
            })